In [2]:
import pandas as pd
import re
import numpy as np
import sys

In [3]:
cig=pd.read_csv("/home/vcth2/" + sys.argv[1], sep="\t", header=None)
cig.columns=["chr", "start", "cigar"]

In [4]:
def where_deletions(chr,start,cigar,min_read_length=1000):
    # I did verify that overall this fn churns out deletions with the right coordinates
    # ultimately this should perhaps also be done with insertions

    # This subfn finds the deletions, sums all previous cigar numbers to get the start position (within the read), and then adds the chromosome and read starts
    def coords(numbers,index, chr, read_start):
        start=np.sum(numbers[0:index]) + read_start - 1
        end=start+numbers[index]
        return[chr,start, end]
    
    # Splitting the cigar string either into one list of numbers and one list of letters, where corresponding numbers/letters have the same index
    numbers=re.split(r'[A-Z]', cigar)
    letters=re.split(r'[0-9]+', cigar)
    numbers.pop(-1)
    letters.pop(0)
    numbers=[int(x) for x in numbers]

    if sum(numbers)> min_read_length:
        
        # Now using numpy to find insertions. I want to remove insertions because they will inflate genomic coordinates relative to the reference genome
        letters=np.array(letters)
        numbers=np.array(numbers)
        Is=np.where((letters=="I") | (letters=="S"))[0].tolist()
        letters=np.delete(letters,Is)
        numbers=np.delete(numbers,Is)
        
        # Now I just need to apply it to every deletion
        deletion_bed=pd.DataFrame([coords(numbers, x, chr, start) for x in np.where(letters=="D")[0].tolist()])
        return(deletion_bed)